In [ ]:
import numpy as np
import time
import cv2
import imutils
from imutils.video import FPS
from imutils.video import VideoStream
import torch

##Individual Blocks

In [ ]:
model = torch.hub.load('/content/drive/MyDrive/TYOLO/yolov5', 'custom', path='/content/drive/MyDrive/TYOLO/yolov5/runs/train/exp16/weights/best.pt', source='local')

In [ ]:
def calculate_distance(y1,y2) :
  # diag = ((x1-x2)**2 + (y1-y2)**2)**(1/2)
  X = 1.0688
  f = 12
  real_h = 1067
  image_h = 480
  obj_h = abs(y1-y2)
  Sensor_h = 5.29 
  D = (f * image_h * real_h) / (obj_h * Sensor_h)
  return str(int(D/1000))

In [ ]:
vs = cv2.VideoCapture('/content/drive/MyDrive/TYOLO/Lara_Test.mp4')
writer = None
(W, H) = (None, None)
# try to determine the total number of frames in the video file
try:
	prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
		else cv2.CAP_PROP_FRAME_COUNT
	total = int(vs.get(prop))
	print("[INFO] {} total frames in video".format(total))
# an error occurred while trying to determine the total
# number of frames in the video file
except:
	print("[INFO] could not determine # of frames in video")
	print("[INFO] no approx. completion time can be provided")
	total = -1

[INFO] 540 total frames in video


In [ ]:
count = 0
while True:
  # read the next frame from the file
  (grabbed, frame) = vs.read()
  if count==0 :
    f = frame
    count+=1
  # if the frame was not grabbed, then we have reached the end
  # of the stream
  if not grabbed:
    break
  # if the frame dimensions are empty, grab them
  if W is None or H is None:
    (H, W) = frame.shape[:2]

  ###################################################################
  # Passing the top 40% frames to the model to improve performance ##

  optimal_frame = np.zeros((frame.shape[0], frame.shape[1],3),dtype=float)
  # print(type(frame[0:int(frame.shape[0]*0.4)][:]))
  optimal_frame[0:int(frame.shape[0]*0.4)][:] = frame[0:int(frame.shape[0]*0.4)][:]
  start = time.time()
  layerOutputs = model(optimal_frame)
  # print(layerOutputs)
  # layerOutputs.print()
  # print(layerOutputs.xyxy[0])
  ####################################################################

  if len(layerOutputs.xyxy[0]) > 0 :
    for i in layerOutputs.xyxy[0] :
      try:
        x1 = i[0]
        y1 = i[1]
        x2 = i[2]
        y2 = i[3]
        label = i[5]
        if(label == 0) :
          label_text = 'go '
        else :
          label_text = 'stop '
        dist = calculate_distance(y1,y2) ## distance calculation using heigth of bounding box
        text =  label_text + dist + ' m'
        print(text)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255,255,255), 2)
        cv2.putText(frame,text, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2) #Put a label over box.
        # print(dist)
        # cv2.putText(frame,, (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
      except:
        continue
  end = time.time()


  if writer is None:
    # initialize our video writer
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter("/content/drive/MyDrive/TYOLO/Test3.mp4", fourcc, 30,
      (frame.shape[1], frame.shape[0]), True)
    # some information on processing single frame
    if total > 0:
      elap = (end - start)
      print("[INFO] single frame took {:.4f} seconds".format(elap))
      print("[INFO] estimated total time to finish: {:.4f}".format(
        elap * total))
  # write the output frame to the disk
  writer.write(frame)
  # release file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

## Combined Function

In [ ]:
def detect_video(yolo_dir_path, yolo_weight_path, input_video_path, output_video_path) :
  print("Loading Model")
  try :
    model = torch.hub.load(yolo_dir_path, 'custom', path=yolo_weight_path, source='local')
    print("Model Successfully loaded")
  except :
    print("Falied to load model")

  ######## Video Detection Part ###################
  vs = cv2.VideoCapture(input_video_path)
  writer = None
  (W, H) = (None, None)
  # try to determine the total number of frames in the video file
  try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
      else cv2.CAP_PROP_FRAME_COUNT
    total = int(vs.get(prop))
    print("[INFO] {} total frames in video".format(total))
  # an error occurred while trying to determine the total
  # number of frames in the video file
  except:
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1


  count = 0
  while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()
    if count==0 :
      f = frame
      count+=1
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
      break
    # if the frame dimensions are empty, grab them
    if W is None or H is None:
      (H, W) = frame.shape[:2]

    ###################################################################
    # Passing the top 40% frames to the model to improve performance ##  

    optimal_frame = np.zeros((frame.shape[0], frame.shape[1],3),dtype=float)
    # print(type(frame[0:int(frame.shape[0]*0.4)][:]))
    optimal_frame[0:int(frame.shape[0]*0.4)][:] = frame[0:int(frame.shape[0]*0.4)][:]
    start = time.time()
    layerOutputs = model(optimal_frame)
    # print(layerOutputs)
    # layerOutputs.print()
    # print(layerOutputs.xyxy[0])
    ####################################################################
    
    if len(layerOutputs.xyxy[0]) > 0 :
      for i in layerOutputs.xyxy[0] :
        try:
          x1 = i[0]
          y1 = i[1]
          x2 = i[2]
          y2 = i[3]
          label = i[5]
          if(label == 0) :
            label_text = 'go '
          else :
            label_text = 'stop '
          dist = calculate_distance(y1,y2)  ## distance calculation
          text =  label_text + dist + ' m'
          print(text)
          cv2.rectangle(frame, (x1, y1), (x2, y2), (255,255,255), 2)
          cv2.putText(frame,text, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2) #Put a label over box.
          # print(dist)
          # cv2.putText(frame,, (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
        except:
          continue
    end = time.time()


    if writer is None:
      # initialize our video writer
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      writer = cv2.VideoWriter(output_video_path, fourcc, 30,
        (frame.shape[1], frame.shape[0]), True)
      # some information on processing single frame
      if total > 0:
        elap = (end - start)
        print("[INFO] single frame took {:.4f} seconds".format(elap))
        print("[INFO] estimated total time to finish: {:.4f}".format(
          elap * total))
    # write the output frame to disk
    writer.write(frame)
    # release the file pointers
  print("[INFO] cleaning up...")
  writer.release()
  vs.release()

## Run for a Video

In [ ]:
yolov5_dir_path = "/content/drive/MyDrive/TYOLO/yolov5"
custom_wt_path = "/content/drive/MyDrive/TYOLO/yolov5/runs/train/exp16/weights/best.pt"
test_video_path = "/content/drive/MyDrive/TYOLO/dataset/test_videos/Test4.mp4"
output_video_path = "/content/drive/MyDrive/TYOLO/dataset/test_videos/Test4_result.mp4"

In [ ]:
detect_video(yolov5_dir_path, custom_wt_path, test_video_path , output_video_path)